In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch import Tensor
from torch.nn.utils.rnn import pack_sequence, pad_sequence, pack_padded_sequence, pad_packed_sequence, PackedSequence

In [2]:
class VaryingLengthDataset(Dataset):
    def __init__(self, n, dmin, dmax):
        super().__init__()
        self.size = n
        self.data = [torch.randint(0, 10, (np.random.randint(dmin, dmax), 2)) for _ in range(n)]
        
    # a commemt
    def __len__(self):
        return self.size
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __repr__(self):
        return "".join(repr(x.shape) for x in self.data)
    
def myfun():
    pass

In [86]:
start = pandas.Timestamp("today")
stop = now + pandas.Timedelta("24h")
freq = pandas.Timedelta("1s")

In [2]:
import tsdm

In [128]:
np.insert?

In [112]:
rng = np.random.default_rng()
timestamps = rng.choice(time_range, 10)

In [104]:
Timestamp(datetime.datetime(1950, 1, 1))

In [110]:
np.random.shuffle

In [100]:
import datetime

In [112]:
type(datetime.timedelta(seconds=3))

In [113]:
issubclass(np.timedelta64, datetime.timedelta)

In [108]:
np.array(datetime.timedelta(seconds=5), dtype="timedelta64[s]").dtype

In [75]:
dsets = []

for key, val in InSilicoData.dataset.items():
    ds = df.to_xarray()
    ds = ds.assign_coords(run_id = key)
    dsets.append(ds)

In [77]:
dsets

In [96]:
np.datetime64?

In [91]:
import numpy as np
np.datetime64('now')

In [95]:
np.datetime64(24, 'h')

In [84]:
xarray.concat(dsets, dim="run_id").to_dataframe()

In [38]:
arrays = [xarray.DataArray(df, name=key) for key, df in InSilicoData.dataset.items()  ]

In [67]:
arrays[0]

In [40]:
xarray.merge( arrays)

In [50]:
df = list(InSilicoData.dataset.values())[0]
df.name = "agagaga"

In [26]:
xarray.Dataset.from_dict(InSilicoData.dataset)

In [19]:
dir(VaryingLengthDataset)

### Batching this Dataset won't work:

In [3]:
MyDataset = VaryingLengthDataset(100, 3, 7)

try:
    next(iter(DataLoader(MyDataset, batch_size=10)))
    print(x)
except Exception as E:
    print(E)

### Let's try with a custom collate_fn

In [4]:
np.random.randint(0, len(MyDataset), 20)

In [5]:
batch = [MyDataset[idx] for idx in np.random.randint(0, len(MyDataset), 5)]
batch

In [6]:
pad_sequence(batch, batch_first=True).shape

In [7]:
def collate_list(batch):
    return batch

try:
    x = next(iter(DataLoader(MyDataset, batch_size=5, collate_fn=collate_list)))
    print(x)
except Exception as E:
    print(E)

In [8]:
def collate_packed(batch):
    batch.sort(key=torch.Tensor.__len__, reverse=True)
    return pack_sequence(batch)

try:
    x = next(iter(DataLoader(MyDataset, batch_size=5, collate_fn=collate_packed)))
    print(x)
except Exception as E:
    print(E)

In [9]:
batch = next(iter(DataLoader(MyDataset, batch_size=5, collate_fn=collate_list)))
[x.T for x in batch]

In [10]:
batch_sorted = batch.copy()
batch_sorted.sort(key=torch.Tensor.__len__, reverse=True)
[x.T for x in batch_sorted]

In [11]:
batch_packed = pack_sequence(batch_sorted)
batch_packed.data.T, batch_packed.batch_sizes

In [12]:
batch_pad_packed, lengths = pad_packed_sequence(batch_packed, batch_first=True)
torch.swapaxes(batch_pad_packed, 1, 2), lengths

In [13]:
unpacked_batch = [x[:l].T for x, l in zip(batch_pad_packed, lengths)]
unpacked_batch

In [14]:
def reconstruct_batch(packed: PackedSequence) -> list[Tensor]:
    d = packed.data.shape[-1]
    dtype = packed.data.dtype
    device = packed.data.device
    
    lengths = []
    b0 = 0
    for b in batch_sizes:
        
        lengths += [b]

In [15]:
reconstructed_batch = batch_pad_packed
reconstructed_batch

In [16]:
batch_pad_packed.shape

In [17]:
def unpack(packed):
    pass

In [18]:
a = MyDataset[:5]
a.sort(key=lambda x: len(x))
a